In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd "/content/gdrive/MyDrive"

/content/gdrive/MyDrive


In [ ]:
import pandas as pd

## Combine the dataset

In [ ]:
old  = pd.read_csv("./Dataset/old_data_kaggle.csv")
#new  = pd.read_csv("./Dataset/NN_improve_tweets.csv")
#print(old.shape[0], new.shape[0])

In [ ]:
# Define the label mapping
df2 = new
label_map = {
    'normal': 0,
    'offensive': 1,
    'hateful': 2
}

output_list = []
for i in df2.index:
  if df2.loc[i, 'hate'] == 1 or  df2.loc[i, 'hate'] == 2:
    output_list.append([df2.loc[i, 'text'], 1])
  else:
    output_list.append([df2.loc[i, 'text'], 0])

output_df = pd.DataFrame(output_list, columns=['text', 'hate'])
print(output_df)
new = output_df
print(len(set(output_df.hate.values.tolist())))

                                                    text  hate
0        Uhmm like 6th grade on a corner of a street....     0
1            a) JTP is a douchebag b) Stewart kicks ass!     0
2                                           ditto bitch!     0
3      damn I have to drive my dad to the airport tha...     0
4                                                     :]     0
...                                                  ...   ...
19929                       Happy birthday @paul_lander!     0
19930  RT @MilradJalen: @raesanni I agree it’s awful ...     0
19931  I was called in yesterday at the Guidance Coun...     0
19932                     @Jeicky_the_cat @skwi69 Thanks     0
19933  thinks she found a good group to help her refocus     0

[19934 rows x 2 columns]
2


In [ ]:
# more emphasis on new dataset
import numpy as np 
old_train_df, old_dev_df, old_test_df =  np.split(old.sample(frac=1, random_state=42),[int(.6*len(old)), int(.8*len(old))])
new_train_df, new_dev_df, new_test_df =  np.split(new.sample(frac=1, random_state=42),[int(.6*len(new)), int(.8*len(new))])
def combine_dataset_k(k):
  sentences = old_train_df['tweet'].values.tolist() + old_dev_df['tweet'].values.tolist() 
  for i in range(k):
    sentences += new_train_df['text'].values.tolist()
    sentences += new_dev_df['text'].values.tolist()
  labels = old_train_df['label'].values.tolist() + old_dev_df['label'].values.tolist() 
  print(len(set(labels)))
  for i in range(k):
    labels += new_train_df['hate'].values.tolist()
    labels += new_dev_df['hate'].values.tolist()

  assert len(sentences) == len(labels)
  print(len(set(labels)))
  # construct combined dataframe
  combined = pd.DataFrame(list(zip(sentences, labels)), columns=['tweet', 'hate'])

  return combined

combined = combine_dataset_k(2)
combined

2
2


,tweet,hate
0,@user âmy mom says my smile is captivatingâ...,0
1,"in 3 days i will be meeting my sis-n-law, coun...",0
2,hating the conservative homophobes using this ...,0
3,awee if this doesn't #scream #friday #acewe...,0
4,fathersday #fatherÃ¢ÂÂs #day #god! #ÃÂ« #...,0
...,...,...
57458,Lots of ppl willing to give u that 'punch' you...,0
57459,lmao ikr. I knew you would hate it. I couldn't...,0
57460,dude that sucks man!,0
57461,"And now: Flashback Round! Come back in, elimin...",0


## Prepare dataset

In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.7 MB/s eta 0:00:0

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
# Split the data into training and validation sets
train_df, dev_df =  train_test_split(combined, test_size=0.2)
print(train_df.shape, dev_df.shape)

(45970, 2) (11493, 2)


In [ ]:
#create custom dataset 
import torch
from torch.utils.data import Dataset
class TweetDataset(Dataset):

    def __init__(self, encodings, labels):
      self.encodings = encodings
      self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['label'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
MAX_LENGTH = 128
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base', use_fast=True)

TEXT_COL = "tweet"
LABEL_COL = "hate"

def create_dataset_improve(dataframe, TEXT_COL, LABEL_COL):
  inputs = {
          "input_ids":[],
          "attention_mask":[]
        }

  sents = dataframe[TEXT_COL].values.tolist()
  for sent in sents:
    tokenized_input = tokenizer(sent,max_length=MAX_LENGTH, padding='max_length', truncation = True)
    inputs["input_ids"].append(torch.tensor(tokenized_input["input_ids"]))
    inputs["attention_mask"].append(torch.tensor(tokenized_input["attention_mask"]))
  # Create a TensorDataset from the input data
  labels = torch.tensor(dataframe[LABEL_COL].values.tolist())
  return TweetDataset(inputs, labels)

train_dataset = create_dataset_improve(train_df, TEXT_COL, LABEL_COL)
dev_dataset = create_dataset_improve(dev_df, TEXT_COL, LABEL_COL)


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Hyperparameter tuning

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.8 MB/s eta 0:00:00


In [ ]:
from sklearn.utils import compute_class_weight
import torch.nn as nn
from transformers import Trainer, TrainingArguments
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
    
def get_class_weights(dataframe,LABEL_COLUMN):
  """computes the class weight and returns a list to account for class imbalance """
  labels = torch.tensor(dataframe[LABEL_COLUMN].values.tolist())
  class_weights=compute_class_weight( class_weight ='balanced',classes = np.unique(labels),y = labels.numpy())
  class_weight_dict = dict(zip(np.unique(labels), class_weights))
  total_class_weights =[]
  for i in range(2):
    if i not in class_weight_dict:
      total_class_weights.append(1) #class_weight 1 for unseen labels
    else:
      total_class_weights.append(class_weight_dict[i])
  total_class_weights =torch.tensor(total_class_weights,dtype=torch.float).to(device)
  return total_class_weights

def create_custom_trainer(class_weights):
  """creates custom trainer that accounts for class imbalance"""
  class CustomTrainer(Trainer):
      def compute_loss(self, model, inputs, return_outputs=False):
          labels = inputs.get("labels")
          # forward pass
          outputs = model(**inputs)
          logits = outputs.get("logits")
          # compute custom loss 
          loss_fct = nn.CrossEntropyLoss(weight=class_weights)
          loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
          return (loss, outputs) if return_outputs else loss
  return CustomTrainer
class_weights = get_class_weights(train_df,'hate')
CustomTrainer = create_custom_trainer(class_weights)

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
from datasets import load_metric
f1_metric =load_metric("f1")
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=1)
  return  f1_metric.compute(predictions=predictions, references=labels)

<ipython-input-12-ba66310c2c0b>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  f1_metric =load_metric("f1")


In [ ]:
import sklearn

from sklearn.metrics import accuracy_score   
from transformers import Trainer, TrainingArguments

import datetime
from datetime import datetime
import optuna

num_labels = 2 # binary classification
model = BertForSequenceClassification.from_pretrained('vinai/bertweet-base',num_labels = num_labels)


def objective(trial):
    # Define hyperparameters to tune
    learning_rate = trial.suggest_float("learning_rate", 1e-5,5e-5, log=True)
    num_train_epochs = trial.suggest_int("num_train_epochs", 3,5)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32])
    per_device_eval_batch_size = per_device_train_batch_size 
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 0.1, log=True)
    output_dir = "./results/"+str(datetime.now())
    # Define training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        push_to_hub=False,
        logging_dir="./logs",
    )

    # Train the model
    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
        compute_metrics=compute_metrics
    )


    trainer.train()
    output = trainer.predict(dev_dataset)
    predictions = np.argmax(output.predictions, axis=1)
    f1 =  f1_metric.compute(predictions=predictions, references=output.label_ids)['f1']
    print(f1)
    return f1

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing BertForSequenceClassification: ['roberta.encoder.layer.11.output.dense.bias', 'roberta.encoder.layer.4.intermediate.dense.bias', 'roberta.encoder.layer.1.output.dense.bias', 'roberta.encoder.layer.7.output.LayerNorm.weight', 'roberta.encoder.layer.8.attention.output.dense.weight', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.encoder.layer.1.output.dense.weight', 'roberta.encoder.layer.11.output.dense.weight', 'roberta.encoder.layer.9.attention.self.value.weight', 'roberta.encoder.layer.9.attention.self.key.weight', 'roberta.encoder.layer.8.attention.self.query.weight', 'roberta.encoder.layer.1.output.LayerNorm.bias', 'roberta.encoder.layer.2.output.dense.bias', 'roberta.encoder.layer.3.output.LayerNorm.bias', 'roberta.encoder.layer

Epoch,Training Loss,Validation Loss,F1
1,0.808200,0.679745,0.541445
2,0.621500,0.836770,0.586626
3,0.497800,0.777743,0.637943


[I 2023-05-01 22:20:21,868] Trial 0 finished with value: 0.6379430159833217 and parameters: {'learning_rate': 1.425059716044744e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 8, 'weight_decay': 3.091922553139464e-05}. Best is trial 0 with value: 0.6379430159833217.


0.6379430159833217


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.518200,0.596182,0.660782
2,0.394200,0.980022,0.620321
3,0.240300,0.853932,0.664430
4,0.222800,0.962083,0.666222


[I 2023-05-01 23:26:35,552] Trial 1 finished with value: 0.6662216288384514 and parameters: {'learning_rate': 2.069437895812307e-05, 'num_train_epochs': 4, 'per_device_train_batch_size': 8, 'weight_decay': 2.0958157107056648e-05}. Best is trial 1 with value: 0.6662216288384514.


0.6662216288384514


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
[W 2023-05-01 23:26:36,504] Trial 2 failed with parameters: {'learning_rate': 1.5822685338586783e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 32, 'weight_decay': 0.0010862951425673576} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 14.75 GiB total capacity; 13.31 GiB already allocated; 28.81 MiB free; 13.68 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.p

OutOfMemoryError: ignored

In [ ]:
# train with best params
import datetime
from datetime import datetime
from transformers import Trainer, TrainingArguments
from datasets import load_metric
f1_metric =load_metric("f1")
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=1)
  return  f1_metric.compute(predictions=predictions, references=labels)
# Define the training arguments

#
learning_rate = 2.069437895812307e-05
epochs = 5
batch_size = 16
#epochs = 3
#batch_size = 16
#learning_rate = 5e-5
output_dir = "./results_combined/"+str(datetime.now())

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate = learning_rate,
    weight_decay=2.0958157107056648e-05,
    logging_dir='./logs',
    logging_steps=1000,
    evaluation_strategy="epoch",
     metric_for_best_model = 'f1',
    do_train = True,
  do_eval = True
)

num_labels = 2 # binary classification
model = BertForSequenceClassification.from_pretrained('vinai/bertweet-base',num_labels = num_labels)

# Tell pytorch to run this model on the GPU.
# Define the trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics
)

# Fine-tune the model
trainer.train()

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing BertForSequenceClassification: ['roberta.encoder.layer.6.intermediate.dense.bias', 'roberta.encoder.layer.2.attention.output.dense.weight', 'roberta.encoder.layer.5.attention.output.dense.weight', 'roberta.encoder.layer.4.output.LayerNorm.weight', 'roberta.encoder.layer.5.attention.output.LayerNorm.bias', 'roberta.encoder.layer.11.attention.output.dense.bias', 'roberta.encoder.layer.5.attention.self.query.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.4.intermediate.dense.bias', 'roberta.encoder.layer.6.attention.self.query.weight', 'roberta.encoder.layer.2.intermediate.dense.weight', 'roberta.encoder.layer.8.attention.output.dense.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.5.output.dense.weight', 'roberta.encoder.layer.5.attention.self.value.weight', 'roberta.encoder.layer.0.attention.self.query.bias', '

Epoch,Training Loss,Validation Loss,F1
1,0.468000,0.326935,0.643387
2,0.384300,0.460992,0.764194
3,0.290800,0.571301,0.781773
4,0.206400,0.624370,0.798814
5,0.145400,0.549837,0.814513


TrainOutput(global_step=14370, training_loss=0.31098823998649666, metrics={'train_runtime': 5690.4323, 'train_samples_per_second': 40.392, 'train_steps_per_second': 2.525, 'total_flos': 1.5119019018624e+16, 'train_loss': 0.31098823998649666, 'epoch': 5.0})

## See its performance on test dataset 

In [ ]:
DATA  = "./Dataset/davidson_data.csv"
df = pd.read_csv(DATA)
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [ ]:
df = df[['tweet', 'hate_speech']]
df

,tweet,hate_speech
0,!!! RT @mayasolovely: As a woman you shouldn't...,0
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0
...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,0
24779,"you've gone and broke the wrong heart baby, an...",0
24780,young buck wanna eat!!.. dat nigguh like I ain...,0
24781,youu got wild bitches tellin you lies,0


In [ ]:
MAX_LENGTH = 128
def create_dataset(dataframe):
  inputs = {
          "input_ids":[],
          "attention_mask":[]
        }

  sents = dataframe['tweet'].values.tolist()
  for sent in sents:
    tokenized_input = tokenizer(sent,max_length=MAX_LENGTH, padding='max_length', truncation = True)
    inputs["input_ids"].append(torch.tensor(tokenized_input["input_ids"]))
    inputs["attention_mask"].append(torch.tensor(tokenized_input["attention_mask"]))

  labels = torch.tensor([0]*dataframe.shape[0])

  return TweetDataset(inputs, labels)

test_dataset = create_dataset(df)

In [ ]:
%ls results_combined/2023-05-01 19:11:05.135270

ls: cannot access 'results_combined/2023-05-01': No such file or directory
ls: cannot access '19:11:05.135270': No such file or directory


In [ ]:
num_labels = 2 # binary classification
path = "./results_combined/2023-05-01 19:11:05.135270/checkpoint-5000"
#path = "./my_models/improved_bertweet"
model = BertForSequenceClassification.from_pretrained(path)

In [ ]:
test_args = TrainingArguments(
    output_dir = "./prediction-results",
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = 32,   
)
test_trainer = CustomTrainer( 
    model=model,
    args=test_args,
    compute_metrics=compute_metrics
)

In [ ]:
output = trainer.predict(test_dataset)
output
# save prediction result 
import numpy as np
#np.save('./combined_1_1_pred_davidson.npy', output.predictions) # save

In [ ]:
import torch.nn.functional as F
probabilities = F.softmax(torch.from_numpy(output.predictions), dim=-1)
pred_labels = np.argmax(output.predictions, axis=1)

In [ ]:
# get the gold labels of the test dataset
gold_labels = []
for x in df['hate_speech'].values.tolist():
  if x == 0 or x == 1:
    gold_labels.append(x)
  else:
    gold_labels.append(1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(gold_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.80      0.87      0.83     19790
           1       0.21      0.14      0.17      4993

    accuracy                           0.72     24783
   macro avg       0.50      0.50      0.50     24783
weighted avg       0.68      0.72      0.70     24783



## Test on old_test

In [ ]:
old_test_df

,id,label,tweet
19727,19728,0,call me crazy- i don't mind but i love to loug...
27408,27409,0,#sukhbir #badal announces office-bearers of ...
29160,29161,0,swim time with zoe!! #family #love #swimming #...
8347,8348,0,i like your smile :) bogumday
15588,15589,0,"i despise hilary and sadly, hearing her speak ..."
...,...,...,...
29802,29803,0,#waltdisneyreso should be ashamed of themselv...
5390,5391,0,@user invited @user over for a catch up and i...
860,861,1,black professor makes assumptions about an ent...
15795,15796,0,#lgbtqhatetrumppay is total #liberal trash. it...


In [ ]:
TEXT_COL = "tweet"
LABEL_COL = "label"
test_dataset = create_dataset_improve(old_test_df, TEXT_COL, LABEL_COL)

In [ ]:
%ls results_combined

'2023-05-01 06:14:19.211303'/


In [ ]:
num_labels = 2 # binary classification
path = "./results_combined/2023-05-01 06:14:19.211303/checkpoint-14000"
#path = "./my_models/improved_bertweet"
model = BertForSequenceClassification.from_pretrained(path)

In [ ]:
test_args = TrainingArguments(
    output_dir = "./prediction-results",
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = 32,   
)
test_trainer = CustomTrainer( 
    model=model,
    args=test_args,
    compute_metrics=compute_metrics
)

In [ ]:
output = trainer.predict(test_dataset)
output

PredictionOutput(predictions=array([[ 3.2945926, -3.4135766],
       [ 3.2925432, -3.4118257],
       [ 3.311343 , -3.4266357],
       ...,
       [-2.8291206,  3.0671194],
       [ 2.5677278, -2.7158272],
       [ 3.188752 , -3.3201377]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 0, 0]), metrics={'test_loss': 0.837080180644989, 'test_f1': 0.6334164588528679, 'test_runtime': 49.5037, 'test_samples_per_second': 129.142, 'test_steps_per_second': 8.08})

In [ ]:
import torch.nn.functional as F
probabilities = F.softmax(torch.from_numpy(output.predictions), dim=-1)
pred_labels = np.argmax(output.predictions, axis=1)

In [ ]:
# get the gold labels of the test dataset
gold_labels = []
for x in old_test_df['label'].values.tolist():
  if x == 0 or x == 1:
    gold_labels.append(x)
  else:
    gold_labels.append(1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(gold_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5934
           1       0.74      0.55      0.63       459

    accuracy                           0.95      6393
   macro avg       0.85      0.77      0.80      6393
weighted avg       0.95      0.95      0.95      6393



## Test on new_test

In [ ]:
new_test_df

,text,hate
17840,Zebra undies #MKR chic in pink dress,1
19822,i just rlly cant wait for all old white ppl to...,0
5162,I should brush my bangs over my eyes and join ...,0
7226,Sneakers or sandals?,0
16631,"RT @Omg_Mitch: I'm not sexist, but sexist joke...",1
...,...,...
11284,"@AnasMechch Well, Baghdadi is also into slaver...",1
11964,@GooseNetworkUSA @06JAnk @PatrickOsgood Give m...,0
5390,I NEED A DAMN IRONMAN IM GONNA CRY!!!!,0
860,fuck you.,0


In [ ]:
TEXT_COL = "text"
LABEL_COL = "hate"
test_dataset = create_dataset_improve(new_test_df, TEXT_COL, LABEL_COL)

In [ ]:
output = trainer.predict(test_dataset)
output

PredictionOutput(predictions=array([[-1.7280347,  1.184792 ],
       [ 3.2206762, -3.3493502],
       [ 3.287317 , -3.4069488],
       ...,
       [ 3.191129 , -3.321773 ],
       [ 3.305617 , -3.4207256],
       [ 2.4323003, -2.5722072]], dtype=float32), label_ids=array([1, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 0.9938700795173645, 'test_f1': 0.6615384615384615, 'test_runtime': 29.7437, 'test_samples_per_second': 134.045, 'test_steps_per_second': 8.405})

In [ ]:
import torch.nn.functional as F
probabilities = F.softmax(torch.from_numpy(output.predictions), dim=-1)
pred_labels = np.argmax(output.predictions, axis=1)

In [ ]:
# get the gold labels of the test dataset
gold_labels = []
for x in new_test_df['hate'].values.tolist():
  if x == 0 or x == 1:
    gold_labels.append(x)
  else:
    gold_labels.append(1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(gold_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.95      0.96      0.96      3511
           1       0.69      0.63      0.66       476

    accuracy                           0.92      3987
   macro avg       0.82      0.80      0.81      3987
weighted avg       0.92      0.92      0.92      3987

